# Video Games Recommender Using Metacritic Review Data

Amongst the many other Recommendation Engines via Python on the internet this is a notebook that takes you through generating recommendations using review data from Metacritic Reviews from publications.

You can access [the original dataset on Kaggle](https://www.kaggle.com/skateddu/metacritic-critic-games-reviews-20112019).
You can access my version which I used in this Notebook [on Kaggle also](https://www.kaggle.com/seyi92coding/metacritic-review-scores-20112019).

And view the [Medium article here](https://rareloot.medium.com/interactive-recommendation-engine-using-metacritic-review-dataset-6cb8d62c76d4).

If you wish to learn how to use Python for similar use cases, I can recommend DataCamp's Recommendation Engine course which only takes 4 hours and you can literally copy their code and make adjustments.




# Upload Modules

As usual, we will need to upload the relevant modules.

In [ ]:
import pandas as pd
import numpy as np
import scipy as sp
from scipy import sparse
from sklearn.metrics.pairwise import cosine_similarity
!pip install fuzzywuzzy
from fuzzywuzzy import fuzz

# Upload and Clean Datasets

If you plan to create your own Recommendation Engine with a different dataset, the 3 most important columns (features) you will need are:

* User ID
* Item ID
* Rating

In this case my dataset includes the name of the item (the video game title) and the name of the user (the publication) so the output can be user-friendly for real-world scenarios.

In [2]:
df = pd.read_csv("/content/Ratings_3col_Metacritic.csv",  error_bad_lines=False, encoding='utf-8') 
df_text = pd.read_csv("/content/Ratings_withText_Metacritic.csv",  error_bad_lines=False, encoding='utf-8')

df.head()

,Unnamed: 0,game_ID,reviewer_ID,console_ID,score
0,0,2423,142,1,100.0
1,1,2423,70,1,100.0
2,2,2423,183,1,100.0
3,3,2423,6,1,100.0
4,4,2423,10,1,100.0


In [3]:
df_text.head()

,Unnamed: 0,game_ID,reviewer_ID,score,game,name,console_ID
0,0,2423,142,100.0,Portal 2,LEVEL (Czech Republic),1
1,1,2423,70,100.0,Portal 2,GameCritics,1
2,2,2423,183,100.0,Portal 2,PC Games (Russia),1
3,3,2423,6,100.0,Portal 2,Adventure Gamers,1
4,4,2423,10,100.0,Portal 2,Armchair Empire,1


In [4]:
#Show an overview of the dataset
df.info()
df_text.info()
#Drop unneccessary columns to speed up processing
df = df.drop(['Unnamed: 0'], axis=1)
df_text = df_text.drop(['Unnamed: 0'], axis=1)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 124267 entries, 0 to 124266
Data columns (total 5 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Unnamed: 0   124267 non-null  int64  
 1   game_ID      124267 non-null  int64  
 2   reviewer_ID  124267 non-null  int64  
 3   console_ID   124267 non-null  int64  
 4   score        124267 non-null  float64
dtypes: float64(1), int64(4)
memory usage: 4.7 MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 124267 entries, 0 to 124266
Data columns (total 7 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Unnamed: 0   124267 non-null  int64  
 1   game_ID      124267 non-null  int64  
 2   reviewer_ID  124267 non-null  int64  
 3   score        124267 non-null  float64
 4   game         124267 non-null  object 
 5   name         124267 non-null  object 
 6   console_ID   124267 non-null  int64  
dtypes: float64(1), int64(4), object(2)
me

In [5]:
#Which columns have null values?
print(df.columns[df.isna().any()].tolist())

#How many null values per column? - Count the missing values in each column
df.isnull().sum()

[]


game_ID        0
reviewer_ID    0
console_ID     0
score          0
dtype: int64

# Formatting Dataset so we can convert into a Pivot Table

This point onwards I will be following the methodlogy [shown in this Kaggle notebook. ](https://www.kaggle.com/yonatanrabinovich/anime-recommendations-project#Cosine-Similarity-Model)

In the immediate steps below:

* We want to only focus on one console type - This should improve the relevancy of the recommendations
* Merge the datasets together into one dataframe
* Remove duplicates
* We want only the main 3 columns.

We will then create a pivot table of our dataset where each row represents all the scores each reviewer has.


In [6]:
#step 1 - Filter to one console type
meta_df = df[df['console_ID']== 1]

#step 2 - Merge on Game ID across both datasets
meta_df = meta_df.merge(df_text, left_on = 'game_ID', right_on = 'game_ID', suffixes= ['_meta', ''])
meta_df.head()

#step 3 - Remove duplicate reviews
def create_uid(row):
  #create unique ID based on reviewer and game
  code = str(row['reviewer_ID']) + "_" + str(row['game_ID'])
  return code
meta_df['uid'] = meta_df.apply(create_uid, axis=1)
#drop duplicates of reviewers who reviewed the same game
meta_df = meta_df.drop_duplicates('uid')
print(meta_df.shape)

#step 3 - Only return the important columns
meta_df = meta_df[['game', 'reviewer_ID', 'score']]

(69398, 10)


In [8]:
#We will create a pivot table of users as rows and games as columns. 
#The pivot table will help us make the calcuations of similarity between the reviewers.
pivot = meta_df.pivot_table(index=['reviewer_ID'], columns=['game'], values='score')
pivot.head()

game,.hack//G.U. Last Recode,0RBITALIS,10 Second Ninja,11-11: Memories Retold,1954: Alcatraz,1979 Revolution: Black Friday,2Dark,39 Days to Mars,4PM,60 Parsecs!,7554,8-Bit Armies,80 Days (2015),911 Operator,A Bird Story,A Case of Distrust,A City Sleeps,A Fistful of Gun,A Game of Dwarves,A Game of Thrones: Genesis,A Golden Wake,A Hat in Time,A New Beginning: Final Cut,A Normal Lost Phone,A Pixel Story,A Story About My Uncle,A Total War Saga: Thrones of Britannia,A Valley Without Wind,A Virus Named TOM,A Way Out,ABZU,ADR1FT,AER: Memories of Old,APB: Reloaded,AR-K Episode 3: The Great Escape,ARK: Survival Evolved,Aarklash: Legacy,Aaru's Awakening,Absolver,Abyss Odyssey,...,Worms W.M.D,Wreckfest,Wuppo,X Rebirth,X4: Foundations,XCOM 2,XCOM 2: War of the Chosen,XCOM: Enemy Unknown,XCOM: Enemy Within,Xanadu Next,Xenonauts,Xotic,Yaiba: Ninja Gaiden Z,Yakuza 0,Yatagarasu: Attack on Cataclysm,Year Walk,Yesterday,Yesterday Origins,Yoku's Island Express,Yonder: The Cloud Catcher Chronicles,Yooka-Laylee,Youropa,Ys Origin,Ys Seven,Ys VI: The Ark of Napishtim,Ys: Memories of Celceta,Ys: The Oath in Felghana,ZHEROS,Zack Zero,Zeno Clash II,Zenzizenzic,Zero Escape: Zero Time Dilemma,Zombeer,Zombi,Zombie Army Trilogy,Zombie Night Terror,Zwei: The Ilvard Insurrection,continue?9876543210,inMomentum,theHunter: Call of the Wild
reviewer_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,75.0,NaN,NaN,75.0,NaN,NaN,70.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,65.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,75.0,NaN,NaN,80.0,NaN,NaN,NaN,NaN,NaN,75.0,NaN,40.0,80.0,75.0,...,75.0,80.0,NaN,35.0,NaN,90.0,90.0,87.0,85.0,NaN,NaN,NaN,NaN,85.0,NaN,NaN,NaN,75.0,80.0,NaN,75.0,80.0,NaN,NaN,NaN,85.0,NaN,NaN,81.0,70.0,NaN,NaN,NaN,65.0,60.0,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,70.0,63.0,80.0,77.0,NaN,38.0,NaN,NaN,65.0,NaN,NaN,83.0,NaN,NaN,NaN,NaN,NaN,75.0,44.0,NaN,82.0,NaN,78.0,74.0,41.0,NaN,70.0,68.0,52.0,NaN,NaN,NaN,NaN,NaN,NaN,59.0,NaN,...,79.0,79.0,NaN,16.0,33.0,87.0,85.0,87.0,90.0,NaN,NaN,NaN,NaN,74.0,NaN,NaN,NaN,71.0,87.0,NaN,82.0,NaN,86.0,NaN,NaN,85.0,NaN,55.0,NaN,50.0,88.0,80.0,NaN,80.0,75.0,NaN,NaN,45.0,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,62.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,55.0,NaN,NaN,65.0,NaN,NaN,NaN,NaN,65.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,90.0,NaN,NaN,NaN,52.0,NaN,NaN,NaN,NaN,50.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Engineer Pivot Table
* Value normalization.

We are using the Normalization Technique called [Linear Scaling](https://developers.google.com/machine-learning/data-prep/transform/normalization) when the feature is more-or-less uniformly distributed across a fixed range. Normalization of ratings means adjusting values measured on different scales to a notionally common scale, often prior to averaging.
* Filling Nan values as 0. 

Most review datasets will be sparse like this one so we need to fill in the NaN values somehow. There could be an argument to use a different method depending on your similarity technique further down the notebook.
* Transposing the pivot for the next step.

Swapping the columns and rows round will allow us to drop the unrated reviews more easily.
* Dropping columns with the values of 0 (unrated).
* Using scipy package to convert to sparse matrix format for the similarity computation.


Once our pivot table is in shape we can run a cosine similarity function that will be turned into a dataframe for us to analyse further.

In [9]:
#Applying lambda function to multiple rows using Dataframe.apply()
#(x-np.mean(x))/(np.max(x)-np.min(x)) = Formula
pivot_n = pivot.apply(lambda x: (x-np.mean(x))/(np.max(x)-np.min(x)), axis=1)

# step 2 - Fill NaNs with Zeros
pivot_n.fillna(0, inplace=True)

# step 3 - Transpose the pivot table
pivot_n = pivot_n.T

# step 4 - Locate the columns that are not zero (unrated)
pivot_n = pivot_n.loc[:, (pivot_n != 0).any(axis=0)]

# step 5 - Create a sparse matrix based on our pivot table
piv_sparse = sp.sparse.csr_matrix(pivot_n.values)

# Cosine Similarity Model

We can calculate the cosine similarity of the games based on what ratings they reviewed from reviewers. The closer they are, the more likely they were rated similarly by the reviewers.

In [10]:
#Compute cosine similarity between samples in X and Y.
game_similarity = cosine_similarity(piv_sparse)

#Turn our similarity kernel matrix into a dataframe
game_sim_df = pd.DataFrame(game_similarity, index = pivot_n.index, columns = pivot_n.index)

In [11]:
game_sim_df.head()

game,.hack//G.U. Last Recode,0RBITALIS,10 Second Ninja,11-11: Memories Retold,1954: Alcatraz,1979 Revolution: Black Friday,2Dark,39 Days to Mars,4PM,60 Parsecs!,7554,8-Bit Armies,80 Days (2015),911 Operator,A Bird Story,A Case of Distrust,A City Sleeps,A Fistful of Gun,A Game of Dwarves,A Game of Thrones: Genesis,A Golden Wake,A Hat in Time,A New Beginning: Final Cut,A Normal Lost Phone,A Pixel Story,A Story About My Uncle,A Total War Saga: Thrones of Britannia,A Valley Without Wind,A Virus Named TOM,A Way Out,ABZU,ADR1FT,AER: Memories of Old,APB: Reloaded,AR-K Episode 3: The Great Escape,ARK: Survival Evolved,Aarklash: Legacy,Aaru's Awakening,Absolver,Abyss Odyssey,...,Worms W.M.D,Wreckfest,Wuppo,X Rebirth,X4: Foundations,XCOM 2,XCOM 2: War of the Chosen,XCOM: Enemy Unknown,XCOM: Enemy Within,Xanadu Next,Xenonauts,Xotic,Yaiba: Ninja Gaiden Z,Yakuza 0,Yatagarasu: Attack on Cataclysm,Year Walk,Yesterday,Yesterday Origins,Yoku's Island Express,Yonder: The Cloud Catcher Chronicles,Yooka-Laylee,Youropa,Ys Origin,Ys Seven,Ys VI: The Ark of Napishtim,Ys: Memories of Celceta,Ys: The Oath in Felghana,ZHEROS,Zack Zero,Zeno Clash II,Zenzizenzic,Zero Escape: Zero Time Dilemma,Zombeer,Zombi,Zombie Army Trilogy,Zombie Night Terror,Zwei: The Ilvard Insurrection,continue?9876543210,inMomentum,theHunter: Call of the Wild
game,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
.hack//G.U. Last Recode,1.000000,0.042438,0.233038,-0.017501,0.126048,0.041152,0.009988,0.035155,-0.005134,-0.000184,0.000000,-0.048852,-0.025492,-0.015058,0.003680,-0.004842,0.195598,0.079491,0.089388,0.107550,0.021228,-0.141094,0.209347,-0.056110,-0.011464,-0.113754,0.010465,0.045323,-0.049397,-0.177245,-0.094799,-0.048729,0.086498,0.007123,-0.007919,0.029659,-0.004733,0.081672,-0.219294,0.007918,...,-0.024060,0.102995,-0.101646,0.113243,0.058762,-0.022877,-0.055166,-0.105248,-0.044730,-0.202572,-0.073857,-0.149573,0.000000,-0.063947,0.041962,-0.066283,-0.008699,-0.005011,-0.235164,0.071097,0.006349,-0.066589,-0.022497,-0.417454,-0.147703,-0.194279,0.016032,0.164973,0.093048,0.311826,-0.215586,-0.113702,0.213879,0.111612,0.086795,-0.044311,0.126856,0.231111,-0.078490,0.049206
0RBITALIS,0.042438,1.000000,0.000000,0.026605,0.076284,0.081384,0.059334,0.055872,0.002434,0.270248,0.000000,-0.064878,0.057377,-0.108111,-0.049412,0.000000,0.079250,0.451131,0.062854,0.180216,-0.054101,-0.136141,0.047461,0.021451,-0.564828,-0.004039,0.033037,0.010461,0.046718,-0.168828,0.089005,0.158373,0.026463,0.000000,-0.235173,0.038153,0.034898,-0.150572,0.035794,0.047849,...,-0.129493,-0.057670,0.032800,0.046809,-0.002866,-0.134875,0.035471,-0.066817,-0.089096,0.038336,-0.003895,-0.032110,0.000000,-0.016153,-0.007754,-0.003161,-0.004273,0.026825,-0.145789,0.056989,-0.043128,-0.162838,-0.104258,-0.131433,-0.008396,-0.082648,0.000000,0.066048,0.235484,0.118511,0.023916,-0.141682,-0.030743,0.114591,-0.012021,0.014795,-0.268087,0.038391,0.000000,0.000000
10 Second Ninja,0.233038,0.000000,1.000000,-0.026061,0.130560,0.005715,-0.044981,0.000000,-0.057328,0.000000,0.000000,0.000000,0.000000,0.000000,0.042388,0.088867,0.000000,0.000000,0.000000,0.114329,0.067358,-0.062720,0.016432,0.014183,0.016648,0.045756,-0.017013,-0.052384,-0.078368,0.077723,-0.089993,-0.014222,-0.007828,-0.043145,0.000000,0.006574,0.009086,0.201026,-0.023978,-0.045288,...,-0.011470,0.086718,-0.030227,0.017967,0.000000,-0.041288,0.007836,-0.031104,0.096962,-0.098675,-0.007936,0.000000,0.000000,-0.053703,-0.002873,-0.066995,-0.036690,-0.022917,-0.126839,0.200078,-0.054871,-0.002338,-0.067696,-0.348066,0.000000,-0.083462,-0.114025,0.274303,0.227464,0.221706,-0.261862,-0.037572,0.133678,0.134163,0.029344,0.072517,0.000000,-0.057384,-0.044667,-0.002237
11-11: Memories Retold,-0.017501,0.026605,-0.026061,1.000000,0.082809,0.229304,-0.128619,0.014383,0.042880,-0.026357,0.164046,-0.019329,-0.042759,0.279633,0.008619,-0.016174,0.159196,-0.259731,0.015250,-0.051062,0.010194,0.062577,-0.103144,0.191438,-0.106

# Text Matching

It is very unlikely anyone is going to input the exact keywords to make a recommendation on so we need to match the user input statement to the most likely item in our dataset.

We do this by:

* Creating a function that matches 2 inputs based on their "Levenshtein Distance"
* Creating a new dataframe containing: Game Title, Game ID
* Creating a function that returns the Game Title based on the ID submitted


Then the function that brings it all together by:

* Assigning a matching score for each game title vs user input
* Sorts by highest matching score to lowest
* Identifies the highest score and the game title
* Returns both back to user

In [13]:
# create a function to find the closest title
def matching_score(a,b):
  #fuzz.ratio(a,b) calculates the Levenshtein Distance between a and b, and returns the score for the distance
   return fuzz.ratio(a,b)
   # exactly the same, the score becomes 100

df_game_names = df_text[['game','game_ID']].drop_duplicates(subset=['game_ID']).set_index('game_ID')
df_game_names = df_text.set_index('game_ID')

# a function to convert index to title
def get_title_from_index(index):
  return df_game_names.iloc[index]['game']

# a function to return the most similar title to the words a user type
def find_closest_title(title):
  #matching_score(a,b) > a is the current row, b is the title we're trying to match
   leven_scores = list(enumerate(df_game_names['game'].apply(matching_score, b=title)))
   sorted_leven_scores = sorted(leven_scores, key=lambda x: x[1], reverse=True)
   closest_title = get_title_from_index(sorted_leven_scores[0][0])
   distance_score = sorted_leven_scores[0][1]
   return closest_title, distance_score
   # Bejeweled Twist, 100

#Check matching works
get_title_from_index(20)
title, distance_score = find_closest_title('Dragonball Z')
print(title)

Dragon Ball FighterZ


# Recommendation Function

Firstly it pulls the closest matching available name based on the input then orders the similarity dataframe by that game title.

In [15]:
def game_recommendation(game):
    #Insert closest title here
    game, distance_score = find_closest_title(game)
    #Counter for Ranking
    number = 1
    print('Recommended because you played {}:\n'.format(game))
    
    for n in game_sim_df.sort_values(by = game, ascending = False).index[1:6]:
        print("#" + str(number) + ": " + n + ", " + str(round(game_sim_df[game][n]*100,2)) + "% " + "match")
        number +=1  

In [16]:
game_recommendation('Yakuza 0')

Recommended because you played Yakuza 0:

#1: Valkyria Chronicles 4, 57.85% match
#2: NieR: Automata, 56.95% match
#3: Dark Souls III, 52.59% match
#4: Monster Hunter: World, 52.53% match
#5: Shadow Tactics: Blades of the Shogun, 52.21% match


# Make Recommender Interactive with Gradio

What would recommenders be without users to recommend to? 

With Gradio we can make the input more easier for anyone to use and even hypertune some of the parameters.

If you want to know more about the cool stuff you can do with Gradio, [visit their website](https://gradio.app/working_with_ml/).


In [ ]:
!pip install gradio

In [18]:
import gradio as gr

In [ ]:
recommender_interface = gr.Interface(game_recommendation, ["text"], 
                           ["text"], title="Top 5 Game Recommendations", description="This is a Recommendation Engine based on how Metacritic professional reviewers have scored games up to 2019 (apologies for the out of date data). Simply input a game you have enjoyed playing and it should return 5 games that have been rated similarily")

recommender_interface.launch(debug=True)

If you want to build your own recommendation engine with Gradio, feel free to borrow this as a starting point. Also you want some help walking through the steps of building one, then I can recommend these Datacamp courses.

* [Building Recommendation Engines in Python](https://datacamp.pxf.io/MXkjON)
* [Building Recommendation Engines with PySpark](https://datacamp.pxf.io/KeP9bz)
